In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

In [ ]:
train = pd.read_csv("../input/loan-prediction-analytics-vidhya/train_ctrUa4K.csv")
test_og = pd.read_csv("../input/loan-prediction-analytics-vidhya/test_lAUu6dG.csv")
test = pd.read_csv("../input/loan-prediction-analytics-vidhya/test_lAUu6dG.csv")

##  <center> **EDA**

In [ ]:
train

In [ ]:
train.drop(['Loan_ID'],axis = 1,inplace = True)
test.drop(['Loan_ID'],axis = 1,inplace = True)

In [ ]:
train.shape

In [ ]:
train.replace("3+",3,inplace = True)
test.replace("3+",3,inplace = True)

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.isnull().sum()

In [ ]:
#Replacing Null Values
train['Gender'] = train['Gender'].fillna(train['Gender'].mode()[0])
train['Married'] = train['Married'].fillna(train['Married'].mode()[0])
train['Self_Employed'] = train['Self_Employed'].fillna(train['Self_Employed'].mode()[0])
train['Dependents'] = train['Dependents'].fillna(train['Dependents'].mode()[0])
train['LoanAmount'] = train['LoanAmount'].fillna(train['LoanAmount'].mean())
train['Loan_Amount_Term'] = train['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mean())
train['Credit_History'] = train['Credit_History'].fillna(train['Credit_History'].mean())

In [ ]:
train.isnull().sum()

In [ ]:
train["Dependents"] = train["Dependents"].astype(str).astype(int)      #converting to integer

In [ ]:
train.shape

## <center> **Visualization**

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot("Gender",data = train,hue = "Loan_Status")
print(train.groupby("Gender")["Loan_Status"].count())

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot("Dependents",data = train,hue = "Loan_Status")
print(train.groupby("Dependents")["Loan_Status"].count())

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(("Education"),data = train,hue = "Loan_Status")
print(train.groupby("Education")["Loan_Status"].count())

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(("Property_Area"),data = train,hue = "Loan_Status")
print(train.groupby("Property_Area")["Loan_Status"].count())

In [ ]:
plt.figure(figsize = (10,6))
corr = train.corr()
sns.heatmap(corr,annot = True)

#### We can see there is correlation between `LoanAmount` and `ApplicantIncome`
#### which means if `applicant income` is high `loan amount` issued is high

## <center> **Checking Test Data**

In [ ]:
test

In [ ]:
test.shape

In [ ]:
test.info()

In [ ]:
test.describe()

In [ ]:
test.isnull().sum()

In [ ]:
#Replacing Null Values
test['Gender'] = test['Gender'].fillna(test['Gender'].mode()[0])
test['Married'] = test['Married'].fillna(test['Married'].mode()[0])
test['Self_Employed'] = test['Self_Employed'].fillna(test['Self_Employed'].mode()[0])
test['Dependents'] = test['Dependents'].fillna(test['Dependents'].mode()[0])
test['LoanAmount'] = test['LoanAmount'].fillna(test['LoanAmount'].mean())
test['Loan_Amount_Term'] = test['Loan_Amount_Term'].fillna(test['Loan_Amount_Term'].mean())
test['Credit_History'] = test['Credit_History'].fillna(test['Credit_History'].mean())

In [ ]:
test["Dependents"] = test["Dependents"].astype(str).astype(int)

In [ ]:
test.isnull().sum()

In [ ]:
test.shape

## <center> **Data Preprocessing**

In [ ]:
#Encoding Categorical Values of train data
le = LabelEncoder()
train["Gender"] = le.fit_transform(train["Gender"])
train["Married"] = le.fit_transform(train["Married"])
train["Education"] = le.fit_transform(train["Education"])
train["Self_Employed"] = le.fit_transform(train["Self_Employed"])
train["Property_Area"] = le.fit_transform(train["Property_Area"])
train["Loan_Status"] = le.fit_transform(train["Loan_Status"])

In [ ]:
train

In [ ]:
# Label Encoding Categorical Values of test data
test["Gender"] = le.fit_transform(test["Gender"])
test["Married"] = le.fit_transform(test["Married"])
test["Education"] = le.fit_transform(test["Education"])
test["Self_Employed"] = le.fit_transform(test["Self_Employed"])
test["Property_Area"] = le.fit_transform(test["Property_Area"])

In [ ]:
test

In [ ]:
x = train.drop(['Loan_Status'],axis = 1)
y = train['Loan_Status']

In [ ]:
#Splitting train data into test and train
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

## <center> **Random Forest**

In [ ]:
rf = RandomForestClassifier()
model = rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
print("accuracy score is: ",accuracy_score(y_test,y_pred))
print("recall score is: ",recall_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

## <center> **XGBoost**

In [ ]:
xg = XGBClassifier()
xg.fit(x_train, y_train)
y_pred1 = xg.predict(x_test)
print("accuracy score is: ",accuracy_score(y_test,y_pred1))
print("recall score is: ",recall_score(y_test,y_pred1))
print(classification_report(y_test,y_pred1))

## <center> **Naive Bayes**

In [ ]:
nb= MultinomialNB()
nb.fit(x_train,y_train)
y_pred2 = nb.predict(x_test)
print("accuracy score is: ",accuracy_score(y_test,y_pred2))
print("recall score is: ",recall_score(y_test,y_pred2))
print(classification_report(y_test,y_pred2))

## <center> **KNN Classifier**

In [ ]:
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(x_train,y_train)
y_pred3 = knn_classifier.predict(x_test)
print("accuracy score is :",accuracy_score(y_test,y_pred3))
print("recall score is: ",recall_score(y_test,y_pred3))
print(classification_report(y_test,y_pred3))

## <Center> Gradient Boost

In [ ]:
gb = GradientBoostingClassifier()
gb.fit(x_train,y_train)
y_pred4 = gb.predict(x_test)
print("accuracy score is :",accuracy_score(y_test,y_pred4))
print("recall score is: ",recall_score(y_test,y_pred4))
print(classification_report(y_test,y_pred4))

## <center> **CatBoost**

In [ ]:
cb = CatBoostClassifier(learning_rate= 0.0055)
cb.fit(x_train,y_train)
y_pred5 = cb.predict(x_test)
print("accuracy score is :",accuracy_score(y_test,y_pred5))
print("recall score is: ",recall_score(y_test,y_pred4))
print(classification_report(y_test,y_pred5))

In [ ]:
prediction = model.predict(test)

In [ ]:
prediction.shape

In [ ]:
def y_n(x):
    y_n_flag = None
    if x == 0:
        y_n_flag = "N"
    if x == 1:
        y_n_flag = "Y"
    
    return y_n_flag

In [ ]:
test_og

In [ ]:
test_og.drop(test_og.iloc[:,1:12],axis = 1,inplace = True)

In [ ]:
test_og["Loan_Status"]=prediction

In [ ]:
test_og["Loan_Status"]=test_og["Loan_Status"].apply(y_n)

In [ ]:
test_og